In [1]:
import numpy as np
import pandas as pd
import gc
from scipy import stats
import time


训练集

In [14]:
#读数据
user = pd.read_csv('data/train_preliminary/user.csv')
ad = pd.read_csv('data/train_preliminary/ad.csv')
click_log = pd.read_csv('data/train_preliminary/click_log.csv')

In [16]:
#获取ID
train_id = user[['user_id']]
train_id

1571

In [4]:
#合并数据
all_data_df = click_log.merge(user, on='user_id', how='left')
all_data_df = all_data_df.merge(ad, on='creative_id', how='left')
del user, ad, click_log
gc.collect()
all_data_df

,time,user_id,creative_id,click_times,age,gender,ad_id,product_id,product_category,advertiser_id,industry
0,9,30920,567330,1,2,1,504423,30673,3,32638,319
1,65,30920,3072255,1,2,1,2642300,1261,2,6783,6
2,56,30920,2361327,1,2,1,2035918,1261,2,6783,6
3,6,309204,325532,1,6,1,292523,27081,3,32066,242
4,59,309204,2746730,1,6,1,2362208,\N,18,14682,88
...,...,...,...,...,...,...,...,...,...,...,...
30082766,6,30920,228129,1,2,1,204785,\N,18,14681,297
30082767,57,30920,2361397,1,2,1,2035987,1261,2,6783,6
30082768,26,30920,629802,1,2,1,559183,\N,18,14678,26
30082769,86,30920,2713031,1,2,1,2333817,\N,18,32019,25


In [5]:
#定义聚合统计特征函数
def aggregate_features(df_):
    df = df_.copy()
    agg_func = {
        'creative_id' : ['nunique'],
        'click_times' : ['max', 'min', 'mean', 'std', 'count'],
        'ad_id' : ['nunique'],
        'product_id' : ['nunique'],
        'product_category' : ['nunique'],
        'advertiser_id' : ['nunique'],
        'industry' : ['nunique'],
        'time':['nunique']
    }
    
    agg_df = df.groupby('user_id').agg(agg_func).reset_index()
    agg_df.columns =  ['_'.join(i).strip('_') for i in agg_df.columns.values]
    
    return agg_df

In [ ]:
#定义众数特征函数
def mode_features(df_, train_features):
    #creative_id_mode
    mode_df = df_.groupby('user_id')['creative_id'].agg(lambda x : stats.mode(x)[0][0]).reset_index()
    mode_df.columns = ['user_id', 'creatice_id_mode']
    train_features = pd.merge(train_features, mode_df, on='user_id', how='left')
    #ad_id_mode
    mode_df = df_.groupby('user_id')['ad_id'].agg(lambda x : stats.mode(x)[0][0]).reset_index()
    mode_df.columns = ['user_id', 'ad_id_mode']
    train_features = pd.merge(train_features, mode_df, on='user_id', how='left')
    #product_id_mode
    mode_df = df_.groupby('user_id')['product_id'].agg(lambda x : stats.mode(x)[0][0]).reset_index()
    mode_df.columns = ['user_id', 'product_id_mode']
    train_features = pd.merge(train_features, mode_df, on='user_id', how='left')
    #product_category_mode
    mode_df = df_.groupby('user_id')['product_category'].agg(lambda x : stats.mode(x)[0][0]).reset_index()
    mode_df.columns = ['user_id', 'product_category_mode']
    train_features = pd.merge(train_features, mode_df, on='user_id', how='left')
    
    #advertiser_id_mode
    mode_df = df_.groupby('user_id')['advertiser_id'].agg(lambda x : stats.mode(x)[0][0]).reset_index()
    mode_df.columns = ['user_id', 'advertiser_id_mode']
    train_features = pd.merge(train_features, mode_df, on='user_id', how='left')
    
    #industry_mode
    mode_df = df_.groupby('user_id')['industry'].agg(lambda x : stats.mode(x)[0][0]).reset_index()
    mode_df.columns = ['user_id', 'industry_mode']
    train_features = pd.merge(train_features, mode_df, on='user_id', how='left')
    
    return train_features

In [6]:
aggregate_features_df = aggregate_features(all_data_df)

In [7]:
aggregate_features_df

,user_id,creative_id_nunique,click_times_max,click_times_min,click_times_mean,click_times_std,click_times_count,ad_id_nunique,product_id_nunique,product_category_nunique,advertiser_id_nunique,industry_nunique,time_nunique
0,1,12,2,1,1.076923,0.277350,13,12,6,3,12,9,10
1,2,42,2,1,1.022222,0.149071,45,42,20,3,36,15,28
2,3,30,1,1,1.000000,0.000000,30,30,17,6,28,8,23
3,4,29,1,1,1.000000,0.000000,29,29,18,6,26,10,15
4,5,33,2,1,1.030303,0.174078,33,33,7,4,30,18,26
...,...,...,...,...,...,...,...,...,...,...,...,...,...
899995,899996,13,1,1,1.000000,0.000000,14,13,5,3,12,5,12
899996,899997,17,2,1,1.111111,0.323381,18,17,10,4,13,10,14
899997,899998,14,2,1,1.071429,0.267261,14,14,5,4,9,5,10
899998,899999,18,1,1,1.000000,0.000000,22,18,5,7,16,14,17


In [8]:
aggregate_features_df.to_csv('output/features/train/aggregate_features_df.csv', index=False)

In [9]:
del aggregate_features_df
gc.collect()

0

In [21]:
#处理\N值
all_data_df.replace('\\N', 0, inplace=True)
all_data_df['product_id'] = all_data_df['product_id'].astype(int)
all_data_df['industry'] = all_data_df['industry'].astype(int)

In [24]:
mode_features_df = mode_features(all_data_df, train_id)

In [25]:
mode_features_df

,user_id,creatice_id_mode,ad_id_mode,product_id_mode,product_category_mode,advertiser_id_mode,industry_mode
0,1,2456345,2116146,0,2,188,326
1,2,1145367,1003786,0,2,42272,6
2,3,66009,60437,0,2,8371,0
3,4,31070,30773,0,2,8877,0
4,5,24333,24966,0,18,2862,6
...,...,...,...,...,...,...,...
899995,899996,2611415,2247799,0,18,6433,54
899996,899997,103918,94426,1250,2,24274,321
899997,899998,71752,66267,0,18,14681,6
899998,899999,12838,14271,0,18,10690,6


In [26]:
mode_features_df.to_csv('output/features/train/mode_features_df.csv', index=False)

In [27]:
del mode_features_df
gc.collect()

8

In [31]:
del all_data_df, train_id
gc.collect()

1501

测试集

In [32]:
ad = pd.read_csv('data/test/ad.csv')
click_log = pd.read_csv('data/test/click_log.csv')

In [33]:
test_id = click_log[['user_id']].drop_duplicates()
test_id

,user_id
0,3131989
11,3131990
52,3131994
65,3131995
82,3132000
...,...
33585384,3131979
33585421,3131983
33585431,3131984
33585456,3131987


In [34]:
all_data_df = pd.merge(ad, click_log, on='creative_id', how='left')
del ad, click_log
gc.collect()
all_data_df

,creative_id,ad_id,product_id,product_category,advertiser_id,industry,time,user_id,click_times
0,1,1,\N,5,381,78,81,3153317,1
1,1,1,\N,5,381,78,81,3284714,1
2,1,1,\N,5,381,78,81,3491929,1
3,1,1,\N,5,381,78,81,3831238,1
4,1,1,\N,5,381,78,82,3304452,1
...,...,...,...,...,...,...,...,...,...
33585507,4445715,3812197,39063,17,427,\N,91,3697572,1
33585508,4445718,3812200,39287,17,427,\N,91,3306334,1
33585509,4445719,3812201,39287,17,427,\N,91,3692401,1
33585510,4445719,3812201,39287,17,427,\N,91,3933083,1


In [35]:
aggregate_features_df = aggregate_features(all_data_df)
aggregate_features_df

,user_id,creative_id_nunique,click_times_max,click_times_min,click_times_mean,click_times_std,click_times_count,ad_id_nunique,product_id_nunique,product_category_nunique,advertiser_id_nunique,industry_nunique,time_nunique
0,3000001,11,1,1,1.000000,0.000000,11,11,7,4,10,5,7
1,3000002,49,2,1,1.065574,0.249590,61,49,24,5,41,21,37
2,3000003,35,1,1,1.000000,0.000000,36,35,15,5,29,17,23
3,3000004,17,1,1,1.000000,0.000000,20,17,7,3,14,9,15
4,3000005,23,2,1,1.035714,0.188982,28,23,11,2,22,12,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,3999996,65,4,1,1.136364,0.460313,66,65,30,5,52,18,30
999996,3999997,18,1,1,1.000000,0.000000,20,18,8,4,17,15,17
999997,3999998,18,2,1,1.052632,0.229416,19,18,10,5,16,13,18
999998,3999999,79,2,1,1.075000,0.265053,80,79,26,4,63,29,45


In [36]:
aggregate_features_df.to_csv('output/features/test/aggregate_features_df.csv', index=False)
del aggregate_features_df
gc.collect()

0

In [37]:
all_data_df.replace('\\N', 0, inplace=True)
all_data_df['product_id'] = all_data_df['product_id'].astype(int)
all_data_df['industry'] = all_data_df['industry'].astype(int)

In [38]:
mode_features_df = mode_features(all_data_df, test_id)
mode_features_df

,user_id,creatice_id_mode,ad_id_mode,product_id_mode,product_category_mode,advertiser_id_mode,industry_mode
0,3131989,103304,93874,0,2,37513,6
1,3131990,1232695,1078353,0,18,10989,6
2,3131994,143002,128468,23,2,2536,247
3,3131995,32571,31960,0,2,22123,0
4,3132000,1443125,1257879,0,2,10334,6
...,...,...,...,...,...,...,...
999995,3131979,760162,671936,0,18,14682,6
999996,3131983,3507883,3021200,0,18,14681,0
999997,3131984,214243,192622,1634,2,16636,0
999998,3131987,74072,68290,0,18,14681,6


In [39]:
mode_features_df.to_csv('output/features/test/mode_features_df.csv', index=False)
del mode_features_df
gc.collect()

0

In [40]:
del all_data_df, test_id
gc.collect()

0